In [1]:
!pip3 install -r ../requirements.txt

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import pandas as pd
import requests
import json
from datetime import date, datetime
import boto3
from io import StringIO
from dotenv import load_dotenv
import os
import time

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")

In [ ]:
s3_client = boto3.client('s3')

response = s3_client.get_object(Bucket = AWS_S3_BUCKET, Key = 'raw/tracker_player_rayzem_2022-09-26 17:54:44.873258.json')


data = response['Body'].read()


# data

In [21]:
df_profile = pd.read_json(StringIO(data.decode('utf-8')))


df_profile


,data
expiryDate,2022-09-26T20:57:38.7444429+00:00
matches,[{'attributes': {'id': 'f68f3277-b067-4788-b36...
metadata,"{'schema': 'riot-api', 'next': 1}"
paginationType,Next
requestingPlayerAttributes,"{'platformSlug': 'riot', 'platformUserIdentifi..."


In [80]:
df_profile_json = json.loads(data.decode('utf-8'))
df_profile_json = json.dumps(df_profile_json)
# df_profile = pd.read_json(df_profile_json)


# df_profile_json



In [18]:
print(df_profile.columns)

Index(['data'], dtype='object')


In [4]:
data_str = data.decode('utf-8')
data_json = json.loads(data_str)



1

In [41]:

expiryDate : str = data_json["data"]["expiryDate"]
requestingPlayerAttributes : dict = data_json["data"]["requestingPlayerAttributes"]
paginationType : str = data_json["data"]["paginationType"]
metadata : dict = data_json["data"]["metadata"]
matches : list = data_json["data"]["matches"]

data = []

for match in matches:
    # data
    attributes : dict = match["attributes"]
    match_metadata : dict = match["metadata"]
    expiryDate : str = match["expiryDate"]
    
    # more data
    segments : list = match["segments"]
    for segment in segments:
        # data
        segment_type: str = segment["type"]
        attributes : dict = segment["attributes"]
        segment_metadata : dict = segment["metadata"]
        expiryDate : str = segment["expiryDate"]
        
        # more data
        stats : dict = segment["stats"]
        for k, v in stats.items():
            row = {}

            row.update(attributes)
            row.update(match_metadata)
            row["expiryDate"] = expiryDate

            row["segment_type"] = segment_type
            row.update(attributes)
            row.update(segment_metadata)
            row["expiryDate"] = expiryDate

            row["stat"] = k
            row.update(v)

            data.append(row)

len(data)

1700

In [45]:
columns = data[0].keys()

In [48]:
df = pd.DataFrame(data, columns=columns)

df


,platformSlug,platformUserIdentifier,modeKey,modeName,modeImageUrl,modeMaxRounds,isAvailable,timestamp,result,map,...,stat,rank,percentile,displayName,displayCategory,category,metadata,value,displayValue,displayType
0,riot,RayzenSama#6999,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,True,2022-09-24T16:17:53.362+00:00,victory,7eaecc1b-4337-bbf6-6ab9-04b8f06b3319,...,playtime,None,None,Playtime,Combat,combat,{},2263.000000,37m 43s,TimeSeconds
1,riot,RayzenSama#6999,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,True,2022-09-24T16:17:53.362+00:00,victory,7eaecc1b-4337-bbf6-6ab9-04b8f06b3319,...,roundsPlayed,None,None,rounds_played,Combat,combat,{},23.000000,23,Number
2,riot,RayzenSama#6999,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,True,2022-09-24T16:17:53.362+00:00,victory,7eaecc1b-4337-bbf6-6ab9-04b8f06b3319,...,roundsWon,None,None,Rounds Won,Combat,combat,{},13.000000,13,Number
3,riot,RayzenSama#6999,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,True,2022-09-24T16:17:53.362+00:00,victory,7eaecc1b-4337-bbf6-6ab9-04b8f06b3319,...,roundsLost,None,None,Rounds Lost,Combat,combat,{},10.000000,10,Number
4,riot,RayzenSama#6999,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,True,2022-09-24T16:17:53.362+00:00,victory,7eaecc1b-4337-bbf6-6ab9-04b8f06b3319,...,placement,None,None,placement,Combat,combat,{},9.000000,9,Number
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,riot,RayzenSama#6999,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,True,2022-05-18T20:49:52.055+00:00,victory,7eaecc1b-4337-bbf6-6ab9-04b8f06b3319,...,kdRatio,None,None,K/D Ratio,None,None,{},0.909091,0.9,NumberPrecision1
1696,riot,RayzenSama#6999,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,True,2022-05-18T20:49:52.055+00:00,victory,7eaecc1b-4337-bbf6-6ab9-04b8f06b3319,...,scorePerRound,None,None,Avg. Score,None,None,{},161.833333,161.8,NumberPrecision1
1697,riot,RayzenSama#6999,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,True,2022-05-18T20:49:52.055+00:00,victory,7eaecc1b-4337-bbf6-6ab9-04b8f06b3319,...,damagePerRound,None,None,Dmg/round,None,None,{},107.000000,107.0,NumberPrecision1
1698,riot,RayzenSama#6999,bomb,Competitive,https://trackercdn.com/cdn/tracker.gg/valorant...,25,True,2022-05-18T20:49:52.055+00:00,victory,7eaecc1b-4337-bbf6-6ab9-04b8f06b3319,...,headshotsPercentage,None,None,Headshot%,None,None,{},9.259259,9,Number


In [37]:
matches[0]["metadata"]
# [0]["stats"]

{'modeKey': 'bomb',
 'modeName': 'Competitive',
 'modeImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/modes/normal.png',
 'modeMaxRounds': 25,
 'isAvailable': True,
 'timestamp': '2022-09-24T16:17:53.362+00:00',
 'result': 'victory',
 'map': '7eaecc1b-4337-bbf6-6ab9-04b8f06b3319',
 'mapName': 'Ascent',
 'mapImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/maps/ascent.jpg'}